# Models
> Ad hoc collection of models.

In [ ]:
#| default_exp models

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from wafer.basics import *
from wafer.init import default_init, lambda_init

## Utils

In [ ]:
#| export
class Lambda(nn.Module):
    "Turn a torch function into a Moudle."
    def __init__(self, f):
        super().__init__()
        self.f = f
    def forward(self, x): return self.f(x)

## basic

In [ ]:
#| export
class XLinear(nn.Module):
    "KAN-like layer with RBF basis."
    def __init__(self, ni, nb, no, fix_basis=False, actn='tanh'):
        super().__init__()
        
        self.weight = nn.Parameter(torch.randn(1,no,ni,nb) / np.sqrt(no*ni))
        self.actn = get_actn(actn)
        # self.basis = nn.Conv1d(1,nb,1)
        # self.centers = nn.Parameter(torch.linspace(-2,2,nb).reshape(1,1,-1), requires_grad=not fix_basis)
        # self.widths = nn.Parameter(torch.ones(nb) * nb / np.sqrt(2) / 2, requires_grad=not fix_basis)
        # self.basis_weight = nn.Parameter(torch.empty(nb))
        # self.basis_bias   = nn.Parameter(torch.empty(nb))
        self.basis_weight = nn.Parameter(torch.ones(nb).reshape(1,1,-1) * nb/np.sqrt(2)/2, requires_grad=not fix_basis)
        self.basis_bias   = nn.Parameter(torch.linspace(-4,4,nb).reshape(1,1,-1), requires_grad=not fix_basis)
        # default_init(self, normal=False)
        # nn.init.uniform_(self.basis_weight, -3, 3)
        # nn.init.uniform_(self.basis_bias, -3, 3)
    
    def forward(self, x):
        if x.ndim < 2: x = x.unsqueeze(0); flag = True
        else: flag = False
        x = x.unsqueeze(-1)
        # X = self.actn(self.basis(x)).permute([0,2,1])
        # X = torch.exp_(-(self.widths * (x - self.centers))**2)
        X = self.actn(x * self.basis_weight + self.basis_bias)
        X = torch.reshape(X, (X.shape[0], 1, *X.shape[1:]))
        Y = self.weight * X
        return torch.sum(Y, (-2,-1)) if not flag else torch.sum(Y, (-2,-1)).squeeze(0)
    
    def plot_basis(self, idx=0, xl=-2, xu=2):
        fig,ax = plt.subplots(figsize=(3,3))
        l = torch.linspace(xl, xu, 100)
        x = torch.stack([l, l], dim=-1)
        x = x.unsqueeze(-1)
        # x = x.unsqueeze(1)
        with torch.no_grad():
            # X = torch.exp_(-(self.widths * (x - self.centers))**2)
            # X = self.actn(self.basis(x)).permute([0,2,1])
            X = self.actn(x * self.basis_weight + self.basis_bias)
        for i in range(X.shape[-1]):
            ax.plot(l, X[:,idx,i])

In [ ]:
# mm = XLinear(2,8,1,actn=Lambda(lambda o: torch.exp(-o*o)))
mm = nn.Sequential(XLinear(2,8,5), XLinear(5,8,3))
count_params(mm)

232

In [ ]:
# mm.plot_basis(0)

In [ ]:
# %%timeit
xx = torch.randn(2)
mm(xx).shape

torch.Size([3])

## Input convex

In [ ]:
#| export
class FICNN(nn.Module):
    "Fully input-convex NN. Refer to [ICNN](https://arxiv.org/abs/1609.07152)."
    def __init__(self, ni: int,                        # input size
                 nh: Union[int, list[int]],            # hidden size
                 no: int,                              # output size
                 num_layer: int=2,                     # number of layers (include the output layer), if `nh` is a list, then the `num_layer = len(nh) + 1`
                 actn: Union[str, nn.Module]='relu',   # hidden activation
                 out_actn: Union[str, nn.Module]=None, # output activation
                 init_gain: float=1.                   # weight initialization gain
                ):
        super().__init__()
        nhs = [nh] * (num_layer - 1) if isinstance(nh, int) else nh
        self.w_y = nn.ModuleList([nn.Linear(ni, nh) for nh in nhs + [no]])
        self.w_z = nn.ModuleList([nn.Linear(i, j, bias=False) for i,j in zip(nhs, nhs[1:] + [no])])
        self.actn = get_actn(actn, nn.ReLU())
        self.out_actn = get_actn(out_actn)

        # Initialize
        lambda_init(self, lambda w,b: (nn.init.xavier_normal_(w, init_gain), nn.init.zeros_(b)))
        self.weight_constraint()
    
    def weight_constraint(self):
        "Apply nonnegative weight constriant."
        with torch.no_grad():
            for n,p in self.w_z.named_parameters():
                if 'weight' in n:
                    p.clamp_min_(0)
    
    def forward(self, x):
        z = self.w_y[0](x)
        if len(self.w_z) == 0: return self.out_actn(z)
        for wz,wy in zip(self.w_z[:-1], self.w_y[1:-1]):
            z = self.actn(wz(z) + wy(x))
        z = self.out_actn(self.w_z[-1](z) + self.w_y[-1](x))
        return z

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()